![](./img/NLP_banner.jpg)

# 词嵌入与fine-tuning
#### \[稀牛学院 x 网易云课程\]《文本分类深度学习模型与实战》课程资料 by [@龙心尘](https://blog.csdn.net/longxinchen_ml)

很多高阶的深度学习自然语言处理任务，都可以用词向量作为基础。我们课程的很多任务，可以用预训练好的word2vec初始化，接下来进行fine-tuning。如本章的文本分类。
其基本思路是将离散的词嵌入到连续的空间中，并以此作为词的表示输入到下层的任务中去。
![](http://i.stack.imgur.com/fYxO9.png)

## 如何使用?
#### \[稀牛学院 x 网易云课程\]《文本分类深度学习模型与实战》课程资料 by [@龙心尘](https://blog.csdn.net/longxinchen_ml)
* 从头训练 
 * 就像word2vec一样, 这一层是可学习的, 用随机数initialize , 通过BP去调整.
* pre-trained + fine tuning 
 * 用其他网络(如 word2vec) 训练好的现成的词向量, 作为初始化参数, 然后继续学习.
* pre-trained + static 
 * 用其他网络(如 word2vec) 训练好的现成的词向量, 作为初始化参数, 并且这些参数保持固定, 不参与网络的学习.

# 基于卷积神经网络的文本分类
#### \[稀牛学院 x 网易云课程\]《文本分类深度学习模型与实战》课程资料整理 by [@龙心尘](https://blog.csdn.net/longxinchen_ml)

* 参考文章：[《paddlepaddle情感分析》](http://paddlepaddle.org/documentation/docs/zh/1.2/beginners_guide/basics/understand_sentiment/index.html)

## CNN
#### \[稀牛学院 x 网易云课程\]《文本分类深度学习模型与实战》课程资料整理 by [@龙心尘](https://blog.csdn.net/longxinchen_ml)


传统CNN包含卷积层、全连接层等组件，并采用softmax多类别分类器和多类交叉熵损失函数，一个典型的卷积神经网络如图所示，我们先介绍用来构造CNN的常见组件。

<p align="center">
<img src="https://github.com/PaddlePaddle/book/blob/develop/03.image_classification/image/lenet.png?raw=true"><br/>
</p>


- 卷积层(convolution layer): 执行卷积操作提取底层到高层的特征，发掘出图片局部关联性质和空间不变性质。
- 池化层(pooling layer): 执行降采样操作。通过取卷积输出特征图中局部区块的最大值(max-pooling)或者均值(avg-pooling)。降采样也是图像处理中常见的一种操作，可以过滤掉一些不重要的高频信息。
- 全连接层(fully-connected layer，或者fc layer): 输入层到隐藏层的神经元是全部连接的。
- 非线性变化: 卷积层、全连接层后面一般都会接非线性变化层，例如Sigmoid、Tanh、ReLu等来增强网络的表达能力，在CNN里最常使用的为ReLu激活函数。
- Dropout: 在模型训练阶段随机让一些隐层节点权重不工作，提高网络的泛化能力，一定程度上防止过拟合。

另外，在训练过程中由于每层参数不断更新，会导致下一次输入分布发生变化，这样导致训练过程需要精心设计超参数。如2015年Sergey Ioffe和Christian Szegedy提出了Batch Normalization (BN)算法中，每个batch对网络中的每一层特征都做归一化，使得每层分布相对稳定。BN算法不仅起到一定的正则作用，而且弱化了一些超参数的设计。经过实验证明，BN算法加速了模型收敛过程，在后来较深的模型中被广泛使用。

## 文本卷积神经网络（CNN）
#### \[稀牛学院 x 网易云课程\]《文本分类深度学习模型与实战》课程资料整理 by [@龙心尘](https://blog.csdn.net/longxinchen_ml)


卷积神经网络经常用来处理具有类似网格拓扑结构（grid-like topology）的数据。例如，图像可以视为二维网格的像素点，自然语言可以视为一维的词序列。卷积神经网络可以提取多种局部特征，并对其进行组合抽象得到更高级的特征表示。实验表明，卷积神经网络能高效地对图像及文本问题进行建模处理。

卷积神经网络主要由卷积（convolution）和池化（pooling）操作构成，其应用及组合方式灵活多变，种类繁多。本小结我们以如图所示的网络进行讲解：

<p align="center">
<img src="https://github.com/PaddlePaddle/book/blob/develop/05.recommender_system/image/text_cnn.png?raw=true" width = "80%" align="center"/><br/>

</p>

假设待处理句子的长度为$n$，其中第$i$个词的词向量（word embedding）为$x_i\in\mathbb{R}^k$，$k$为维度大小。

首先，进行词向量的拼接操作：将每$h$个词拼接起来形成一个大小为$h$的词窗口，记为$x_{i:i+h-1}$，它表示词序列$x_{i},x_{i+1},\ldots,x_{i+h-1}$的拼接，其中，$i$表示词窗口中第一个词在整个句子中的位置，取值范围从$1$到$n-h+1$，$x_{i:i+h-1}\in\mathbb{R}^{hk}$。

其次，进行卷积操作：把卷积核(kernel)$w\in\mathbb{R}^{hk}$应用于包含$h$个词的窗口$x_{i:i+h-1}$，得到特征$c_i=f(w\cdot x_{i:i+h-1}+b)$，其中$b\in\mathbb{R}$为偏置项（bias），$f$为非线性激活函数，如$sigmoid$。将卷积核应用于句子中所有的词窗口${x_{1:h},x_{2:h+1},\ldots,x_{n-h+1:n}}$，产生一个特征图（feature map）：

$$c=[c_1,c_2,\ldots,c_{n-h+1}], c \in \mathbb{R}^{n-h+1}$$

接下来，对特征图采用时间维度上的最大池化（max pooling over time）操作得到此卷积核对应的整句话的特征$\hat c$，它是特征图中所有元素的最大值：

$$\hat c=max(c)$$

对于一般的短文本分类问题，上文所述的简单的文本卷积网络即可达到很高的正确率。若想得到更抽象更高级的文本特征表示，可以构建深层文本卷积神经网络。



# 基于LSTM的文本分类
#### \[稀牛学院 x 网易云课程\]《文本分类深度学习模型与实战》课程资料整理 by [@龙心尘](https://blog.csdn.net/longxinchen_ml)

## 循环神经网络（RNN）
#### \[稀牛学院 x 网易云课程\]《文本分类深度学习模型与实战》课程资料整理 by [@龙心尘](https://blog.csdn.net/longxinchen_ml)

循环神经网络是一种能对序列数据进行精确建模的有力工具。实际上，循环神经网络的理论计算能力是图灵完备的。自然语言是一种典型的序列数据（词序列），近年来，循环神经网络及其变体（如long short term memory等）在自然语言处理的多个领域，如语言模型、句法解析、语义角色标注（或一般的序列标注）、语义表示、图文生成、对话、机器翻译等任务上均表现优异甚至成为目前效果最好的方法。

<p align="center">
<img src="https://github.com/PaddlePaddle/book/blob/develop/06.understand_sentiment/image/rnn.png?raw=true" width = "60%" align="center"/><br/>
</p>
<p align="center">
<img src="img/rnn1.png" width = "60%" align="center"/><br/>
</p>

循环神经网络按时间展开后如图所示：在第$t$时刻，网络读入第$t$个输入$x_t$（向量表示）及前一时刻隐层的状态值$h_{t-1}$（向量表示，$h_0$一般初始化为$0$向量），计算得出本时刻隐层的状态值$h_t$，重复这一步骤直至读完所有输入。如果将循环神经网络所表示的函数记为$f$，则其公式可表示为：

$$h_t=f(x_t,h_{t-1})=\sigma(W_{xh}x_t+W_{hh}h_{t-1}+b_h)$$

其中$W_{xh}$是输入到隐层的矩阵参数，$W_{hh}$是隐层到隐层的矩阵参数，$b_h$为隐层的偏置向量（bias）参数，$\sigma$为$sigmoid$函数。  

在处理自然语言时，一般会先将词（one-hot表示）映射为其词向量（word embedding）表示，然后再作为循环神经网络每一时刻的输入$x_t$。此外，可以根据实际需要的不同在循环神经网络的隐层上连接其它层。如，可以把一个循环神经网络的隐层输出连接至下一个循环神经网络的输入构建深层（deep or stacked）循环神经网络，或者提取最后一个时刻的隐层状态作为句子表示进而使用分类模型等等。

## 长短期记忆网络（LSTM）
#### \[稀牛学院 x 网易云课程\]《文本分类深度学习模型与实战》课程资料整理 by [@龙心尘](https://blog.csdn.net/longxinchen_ml)


对于较长的序列数据，循环神经网络的训练过程中容易出现梯度消失或爆炸现象。为了解决这一问题，Hochreiter S, Schmidhuber J. (1997)提出了LSTM(long short term memory)。  

相比于简单的循环神经网络，LSTM增加了记忆单元$c$、输入门$i$、遗忘门$f$及输出门$o$。这些门及记忆单元组合起来大大提升了循环神经网络处理长序列数据的能力。若将基于LSTM的循环神经网络表示的函数记为$F$，则其公式为：

$$ h_t=F(x_t,h_{t-1})$$

$F$由下列公式组合而成：
$$ i_t = \sigma{(W_{xi}x_t+W_{hi}h_{t-1}+W_{ci}c_{t-1}+b_i)} $$
$$ f_t = \sigma(W_{xf}x_t+W_{hf}h_{t-1}+W_{cf}c_{t-1}+b_f) $$
$$ c_t = f_t\odot c_{t-1}+i_t\odot tanh(W_{xc}x_t+W_{hc}h_{t-1}+b_c) $$
$$ o_t = \sigma(W_{xo}x_t+W_{ho}h_{t-1}+W_{co}c_{t}+b_o) $$
$$ h_t = o_t\odot tanh(c_t) $$
其中，$i_t, f_t, c_t, o_t$分别表示输入门，遗忘门，记忆单元及输出门的向量值，带角标的$W$及$b$为模型参数，$tanh$为双曲正切函数，$\odot$表示逐元素（elementwise）的乘法操作。输入门控制着新输入进入记忆单元$c$的强度，遗忘门控制着记忆单元维持上一时刻值的强度，输出门控制着输出记忆单元的强度。三种门的计算方式类似，但有着完全不同的参数，它们各自以不同的方式控制着记忆单元$c$，如图所示：

<p align="center">
<img src="https://github.com/PaddlePaddle/book/blob/develop/06.understand_sentiment/image/lstm.png?raw=true" width = "65%" align="center"/><br/>
</p>
<p align="center">
<img src="img/LSTM1.png" width = "60%" align="center"/><br/>
</p>

<p align="center">
<img src="img/LSTM3.jpg" width=1050><br/>
</p>

LSTM通过给简单的循环神经网络增加记忆及控制门的方式，增强了其处理远距离依赖问题的能力。类似原理的改进还有Gated Recurrent Unit (GRU)，其设计更为简洁一些。**这些改进虽然各有不同，但是它们的宏观描述却与简单的循环神经网络一样（如图所示），即隐状态依据当前输入及前一时刻的隐状态来改变，不断地循环这一过程直至输入处理完毕：**

$$ h_t=Recrurent(x_t,h_{t-1})$$

其中，$Recrurent$可以表示简单的循环神经网络、GRU或LSTM。

## RCNN（循环卷积神经网络）
#### \[稀牛学院 x 网易云课程\]《文本分类深度学习模型与实战》课程资料整理 by [@龙心尘](https://blog.csdn.net/longxinchen_ml)

* 参考文章[《Recurrent Convolutional Neural Networks for Text Classification》](http://rsarxiv.github.io/2016/05/27/Recurrent-Convolutional-Neural-Networks-for-Text-Classification-PaperWeekly/)

双向循环神经网络
<p align="center">
<img src="img/bilstm.png" width=450><br/>
</p>
RCNN模型架构图如下：

<p align="center">
<img src="http://rsarxiv.github.io/2016/05/27/Recurrent-Convolutional-Neural-Networks-for-Text-Classification-PaperWeekly/fig1.png" width=950><br/>
</p>

首先，构造CNN的卷积层，卷积层的本质是一个BiRNN模型，通过正向和反向循环来构造一个单词的下文和上文，如下式：

![](http://rsarxiv.github.io/2016/05/27/Recurrent-Convolutional-Neural-Networks-for-Text-Classification-PaperWeekly/fig2.png)

得到单词的上下文表示之后，用拼接的方式来表示这个单词，如下式：

![](http://rsarxiv.github.io/2016/05/27/Recurrent-Convolutional-Neural-Networks-for-Text-Classification-PaperWeekly/fig3.png)

将该词向量放入一个单层神经网络中，得到所谓的潜语义向量（latent semantic vector），这里卷积层的计算结束了，时间复杂度仍是O(n)。接下来进行池化层（max-pooling），即将刚刚得到的所有单词的潜语义向量中每个维度上最大的值选出组成一个新的向量，这里采用max-pooling可以将向量中最大的特征提取出来，从而获取到整个文本的信息。池化过程时间复杂度也是O(n)，所以整个模型的时间复杂度是O(n)。得到文本特征向量之后，进行分类。

# Transformer/selt-attention介绍
#### \[稀牛学院 x 网易云课程\]《文本分类深度学习模型与实战》课程资料整理 by [@龙心尘](https://blog.csdn.net/longxinchen_ml)

Transformer是个叠加的“自注意力机制（Self Attention）”构成的深度网络。其关键是自注意力机制（Self Attention）。所以我们主要介绍selt-attention。

当我们想对句子“he animal didn't cross the street because it was too tired”中“it”这个词编码时，注意力机制的基本思想是认为这个句话中每个词对it的语义均会有贡献。那怎么综合这些贡献呢，就是直接将每个词的embedding向量**加权求和**。
所以关键的问题是如何得到每个词各自的权重，关系更近的词的权重更大。比如这句话中"The Animal"的权重就应该更大，它们的信息应该更多地编码到“it”中。
自注意力机制得到权重的方法非常简单，就是两个词向量的内积。最终通过一个softmax将各个权重归一化。
![](https://jalammar.github.io/images/t/transformer_self-attention_visualization.png)
在上图中，颜色的粗细代表该词的权重大小，权重由该词与“it”的内积得到，最终通过一个softmax将各个权重归一化。

自注意力机制其实是最原始意义的卷积的思想的推广，因为卷积本身就是一种“加权求和”。

* 参考文章：[《The Illustrated Transformer》](https://jalammar.github.io/illustrated-transformer/)
* 参考文章：[《基于Transformer的神经机器翻译》](https://colab.research.google.com/drive/1Wt9Jwynnki6lipwUcy0Sz5WKG7MYSGs0#scrollTo=3twSbimFUgQq)

## 版权归 © 稀牛学院 所有 保留所有权利
![](./img/xiniu_neteasy.png)